##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 Penguin 数据集的简单 TFX 流水线教程

***运行简单 TFX 流水线的简短教程。***

注：我们建议在 Colab 笔记本中运行本教程，无需进行设置！只需单击“在 Google Colab 中运行”

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://tensorflow.google.cn/tfx/tutorials/tfx/penguin_simple"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org上查看</a></td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/penguin_simple.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 运行</a> </td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/penguin_simple.ipynb"><img width="32px" src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tfx/tutorials/tfx/penguin_simple.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a></td>
</table></div>

在这个基于笔记本的教程中，我们将为简单分类模型创建并运行 TFX 流水线。该流水线将包含三个基本的 TFX 组件：ExampleGen、Trainer 和 Pusher。流水线还包括最小的 ML 工作流程，例如导入数据、训练模型和导出经过训练的模型。

请参阅[了解 TFX 流水线](https://tensorflow.google.cn/tfx/guide/understanding_tfx_pipelines)以了解有关 TFX 中各种概念的更多信息。

## 安装

我们首先需要安装 TFX Python 软件包并下载将用于模型的数据集。

### 升级 Pip

为避免在本地运行时升级系统中的 Pip，请检查以确保我们在 Colab 中运行。本地系统当然可以单独升级。

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### 安装 TFX


In [ ]:
!pip install -U tfx

### 卸载 Shapely

TODO(b/263441833) 这是避免 ImportError 的临时解决方案。最终，应该通过支持最新版本的 Bigquery 来处理，而不是卸载其他额外的依赖项。

In [ ]:
!pip uninstall shapely -y

### 是否已重新启动运行时？

如果您使用的是 Google Colab，则在首次运行上面的代码单元时必须重新启动运行时，方法是单击上面的“重新启动运行时”按钮或使用“运行时 &gt; 重新启动运行时...”菜单。这样做的原因是 Colab 加载软件包的方式。

检查 TensorFlow 和 TFX 版本。

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### 设置变量

有一些变量用于定义流水线。您可以根据需要自定义这些变量。默认情况下，流水线的所有输出都将在当前目录下生成。

In [ ]:
import os

PIPELINE_NAME = "penguin-simple"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### 准备示例数据

我们将下载要在我们的 TFX 流水线中使用的示例数据集。我们要使用的数据集为 [Palmer Penguins 数据集](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)，该数据集也在其他 [TFX 示例](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin)中使用。

该数据集中有四个数字特征：

- culmen_length_mm
- culmen_depth_mm
- flipper_length_mm
- body_mass_g

所有特征均已标准化为具有范围 [0,1]。我们将建立一个预测企鹅 `species` 的分类模型。

因为 TFX ExampleGen 从目录中读取输入，所以我们需要创建一个目录并将数据集复制到其中。

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

快速浏览一下 CSV 文件。

In [ ]:
!head {_data_filepath}

您应该能够看到五个值。`species` 为 0、1 或 2，所有其他特征的值应介于 0 和 1 之间。

## 创建流水线

TFX 流水线是使用 Python API 定义的。我们将定义包含以下三个组件的流水线。

- CsvExampleGen：读入数据文件并将其转换为 TFX 内部格式以供进一步处理。有多种 [ExampleGen](https://tensorflow.google.cn/tfx/guide/examplegen)，可用于不同格式。在本教程中，我们将使用接受 CSV 文件输入的 CsvExampleGen。
- Trainer：训练 ML 模型。[Trainer 组件](https://tensorflow.google.cn/tfx/guide/trainer)需要用户提供模型定义代码。您可以使用 TensorFlow API 指定如何训练模型并以 *saved_model* 格式保存它。
- Pusher：将经过训练的模型复制到 TFX 流水线之外。可以将 [Pusher 组件](https://tensorflow.google.cn/tfx/guide/pusher)视为经过训练的 ML 模型的部署过程。

在实际定义流水线之前，我们需要先为 Trainer 组件编写模型代码。

### 编写模型训练代码

我们将使用 TensorFlow Keras API 创建一个简单的 DNN 分类模型。此模型训练代码将保存到单独的文件中。

在本教程中，我们将使用 TFX 的[通用 Trainer](https://tensorflow.google.cn/tfx/guide/trainer#generic_trainer)，它支持基于 Keras 的模型。您需要编写一个包含 `run_fn` 函数的 Python 文件，该函数是 `Trainer` 组件的入口点。

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://tensorflow.google.cn/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

现在，您已完成构建 TFX 流水线的所有准备步骤。

### 编写流水线定义

我们定义一个函数来创建 TFX 流水线。`Pipeline` 对象表示 TFX 流水线，可以使用 TFX 支持的流水线编排系统之一来运行。


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a three component penguin pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

## 运行流水线

TFX 支持使用多种编排器来运行流水线。在本教程中，我们将使用 `LocalDagRunner`，它包含在 TFX Python 软件包，并在本地环境中运行流水线。我们经常将 TFX 流水线称为“DAG”，DAG 代表有向无环图。

`LocalDagRunner` 为开发和调试提供了快速迭代。TFX 还支持其他编排器，包括适用于生产用例的 Kubeflow Pipelines 和 Apache Airflow。

请参阅 [Cloud AI Platform Pipelines 上的 TFX](https://tensorflow.google.cn/tfx/tutorials/tfx/cloud-ai-platform-pipelines) 或 [TFX Airflow 教程](https://tensorflow.google.cn/tfx/tutorials/tfx/airflow_workshop)来详细了解其他编排系统。

现在，我们创建一个 `LocalDagRunner` 并传递一个从我们已经定义的函数创建的 `Pipeline` 对象。

流水线直接运行，您可以查看日志来了解流水线（包括机器学习模型训练）的进度。

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

如果流水线成功完成，您应该在日志末尾看到“INFO:absl:Component Pusher is finished.”。因为 `Pusher` 组件是流水线的最后一个组件。

如果您在之前的步骤中未更改变量，Pusher 组件会将经过训练的模型推送到 `SERVING_MODEL_DIR`，即 `serving_model/penguin-simple` 目录。您可以从 Colab 左侧面板中的文件浏览器查看结果，或者使用以下命令：

In [ ]:
# List files in created model directory.
!find {SERVING_MODEL_DIR}

## 后续步骤

您可以在 https://tensorflow.google.cn/tfx/tutorials 上找到更多资源。

请参阅[了解 TFX 流水线](https://tensorflow.google.cn/tfx/guide/understanding_tfx_pipelines)来详细了解 TFX 中的各种概念。
